In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib
from matplotlib import pyplot as plt
np.random.seed(10)
# importing necessary keras packages
import keras
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Activation
from keras.models import load_model
from keras.optimizers import SGD

/home/daniel/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
Using TensorFlow backend.
/home/daniel/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/daniel/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be 

In [2]:
df = pd.read_csv('cleaned.csv')

In [3]:
df.head()

,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,...,key_C#,key_D,key_D#,key_E,key_F,key_F#,key_G,key_G#,mode_Major,mode_Minor
0,0,0.611,0.389,0.910,0.000,0.3460,-1.828,0.0525,166.969,0.814,...,1,0,0,0,0,0,0,0,1,0
1,1,0.246,0.590,0.737,0.000,0.1510,-5.559,0.0868,174.003,0.816,...,0,0,0,0,0,1,0,0,0,1
2,3,0.952,0.663,0.131,0.000,0.1030,-13.879,0.0362,99.488,0.368,...,0,0,0,0,0,0,0,0,0,1
3,0,0.703,0.240,0.326,0.000,0.0985,-12.178,0.0395,171.758,0.227,...,1,0,0,0,0,0,0,0,1,0
4,4,0.950,0.331,0.225,0.123,0.2020,-21.150,0.0456,140.576,0.390,...,0,0,0,0,1,0,0,0,1,0


In [4]:
#Let's split the dataset into train, validation and test data
train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

In [5]:
# We have plenty of rows in the dataset. Before starting a training  session I would like to 
# make sure that the network is functioning properly.
# So let's just play around with a fraction of the dataset, I don't want to lock my computer for hours
demo_train = train[:10000]
demo_validate = validate[:2000]
demo_test = test[:2000]

In [6]:
#remove 'demo_' to run the training on the whole dataset.

# Let's extract the target column from the dataset.
Y_train = demo_train['popularity'].values
Y_validate = demo_validate['popularity'].values
Y_test = demo_test['popularity'].values

# We also create the train, test, and validation input here
X_train = demo_train.drop(columns=['popularity'])
X_validate = demo_validate.drop(columns=['popularity'])
X_test = demo_test.drop(columns=['popularity'])

In [8]:
# Not all features are standardized so let's do it before we start he training
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_validate = scaler.transform(X_validate)
X_test = scaler.transform(X_test)

In [9]:
#define callbacks
patience=30
early_stopping=EarlyStopping(patience=patience, verbose=1)
checkpointer=ModelCheckpoint(filepath='weights.hdf5', save_best_only=True, verbose=1)

model = Sequential()
model.add(Dense(output_dim=40, input_dim=X_train.shape[1]))
model.add(Activation('relu'))
model.add(Dense(output_dim=30))
model.add(Activation('relu'))
model.add(Dense(output_dim=15))
model.add(Activation('relu'))
# This a regression problem where we target values in 0-100 range
model.add(Dense(output_dim=1, activation='relu'))
# Let's have a look at the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 40)                2040      
_________________________________________________________________
activation_1 (Activation)    (None, 40)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 30)                1230      
_________________________________________________________________
activation_2 (Activation)    (None, 30)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 15)                465       
_________________________________________________________________
activation_3 (Activation)    (None, 15)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                

/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=50, units=40)`
  import sys
/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=30)`
  if __name__ == '__main__':
/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=15)`
  # This is added back by InteractiveShellApp.init_path()
/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=1)`
  


In [10]:
sgd = SGD(lr=0.00001)
# using mse for regression problem
model.compile(loss='mse', optimizer=sgd)
history=model.fit(X_train,Y_train,epochs=200, 
                  batch_size=64,
                  verbose=2,
                  validation_data=(X_validate, Y_validate),
                  callbacks=[checkpointer, early_stopping])


Train on 10000 samples, validate on 2000 samples
Epoch 1/200
 - 0s - loss: 1593.6691 - val_loss: 1465.3513

Epoch 00001: val_loss improved from inf to 1465.35131, saving model to weights.hdf5
Epoch 2/200
 - 0s - loss: 1437.1028 - val_loss: 1235.7442

Epoch 00002: val_loss improved from 1465.35131 to 1235.74421, saving model to weights.hdf5
Epoch 3/200
 - 0s - loss: 1056.5550 - val_loss: 697.9362

Epoch 00003: val_loss improved from 1235.74421 to 697.93624, saving model to weights.hdf5
Epoch 4/200
 - 0s - loss: 473.9921 - val_loss: 294.6849

Epoch 00004: val_loss improved from 697.93624 to 294.68485, saving model to weights.hdf5
Epoch 5/200
 - 0s - loss: 260.7110 - val_loss: 227.3180

Epoch 00005: val_loss improved from 294.68485 to 227.31804, saving model to weights.hdf5
Epoch 6/200
 - 0s - loss: 207.6590 - val_loss: 188.5561

Epoch 00006: val_loss improved from 227.31804 to 188.55607, saving model to weights.hdf5
Epoch 7/200
 - 0s - loss: 174.6854 - val_loss: 162.0999

Epoch 00007: v


Epoch 00056: val_loss improved from 95.84756 to 95.75008, saving model to weights.hdf5
Epoch 57/200
 - 0s - loss: 95.0465 - val_loss: 95.6959

Epoch 00057: val_loss improved from 95.75008 to 95.69593, saving model to weights.hdf5
Epoch 58/200
 - 0s - loss: 94.9251 - val_loss: 95.5944

Epoch 00058: val_loss improved from 95.69593 to 95.59437, saving model to weights.hdf5
Epoch 59/200
 - 0s - loss: 94.8060 - val_loss: 95.5710

Epoch 00059: val_loss improved from 95.59437 to 95.57102, saving model to weights.hdf5
Epoch 60/200
 - 0s - loss: 94.6947 - val_loss: 95.6548

Epoch 00060: val_loss did not improve from 95.57102
Epoch 61/200
 - 0s - loss: 94.5864 - val_loss: 95.4559

Epoch 00061: val_loss improved from 95.57102 to 95.45593, saving model to weights.hdf5
Epoch 62/200
 - 0s - loss: 94.4796 - val_loss: 95.3999

Epoch 00062: val_loss improved from 95.45593 to 95.39988, saving model to weights.hdf5
Epoch 63/200
 - 0s - loss: 94.3720 - val_loss: 95.3559

Epoch 00063: val_loss improved fr


Epoch 00118: val_loss did not improve from 93.12226
Epoch 119/200
 - 0s - loss: 90.6516 - val_loss: 93.0874

Epoch 00119: val_loss improved from 93.12226 to 93.08736, saving model to weights.hdf5
Epoch 120/200
 - 0s - loss: 90.6110 - val_loss: 93.0990

Epoch 00120: val_loss did not improve from 93.08736
Epoch 121/200
 - 0s - loss: 90.5693 - val_loss: 93.0811

Epoch 00121: val_loss improved from 93.08736 to 93.08108, saving model to weights.hdf5
Epoch 122/200
 - 0s - loss: 90.5248 - val_loss: 93.0216

Epoch 00122: val_loss improved from 93.08108 to 93.02158, saving model to weights.hdf5
Epoch 123/200
 - 0s - loss: 90.4810 - val_loss: 93.1018

Epoch 00123: val_loss did not improve from 93.02158
Epoch 124/200
 - 0s - loss: 90.4346 - val_loss: 92.9878

Epoch 00124: val_loss improved from 93.02158 to 92.98780, saving model to weights.hdf5
Epoch 125/200
 - 0s - loss: 90.4000 - val_loss: 93.0317

Epoch 00125: val_loss did not improve from 92.98780
Epoch 126/200
 - 0s - loss: 90.3522 - val_lo

 - 0s - loss: 88.4596 - val_loss: 92.3982

Epoch 00185: val_loss did not improve from 92.38302
Epoch 186/200
 - 0s - loss: 88.4400 - val_loss: 92.3642

Epoch 00186: val_loss improved from 92.38302 to 92.36419, saving model to weights.hdf5
Epoch 187/200
 - 0s - loss: 88.4157 - val_loss: 92.4075

Epoch 00187: val_loss did not improve from 92.36419
Epoch 188/200
 - 0s - loss: 88.3740 - val_loss: 92.4773

Epoch 00188: val_loss did not improve from 92.36419
Epoch 189/200
 - 0s - loss: 88.3630 - val_loss: 92.3758

Epoch 00189: val_loss did not improve from 92.36419
Epoch 190/200
 - 0s - loss: 88.3417 - val_loss: 92.4085

Epoch 00190: val_loss did not improve from 92.36419
Epoch 191/200
 - 0s - loss: 88.3111 - val_loss: 92.3786

Epoch 00191: val_loss did not improve from 92.36419
Epoch 192/200
 - 0s - loss: 88.2802 - val_loss: 92.3475

Epoch 00192: val_loss improved from 92.36419 to 92.34747, saving model to weights.hdf5
Epoch 193/200
 - 0s - loss: 88.2582 - val_loss: 92.3640

Epoch 00193: va

In [12]:
# Load the weights that performed best on the validation dataset
from sklearn.metrics import mean_squared_error
model = load_model('weights.hdf5')
# predictions for the test dataset
preds = model.predict(X_test)
test_err = mean_squared_error(Y_test, preds)
print("Test error: ",test_err)

Test error:  92.49769032881267


Training only on a sample dataset gave us 92.5 error on the test dataset. This means that model's predictions miss the correct value by 9.7 on average. So far not the best result, but still it shows some correspondence between the features and the popularity. Next step is to start investigating what effect the model has on the result and wehter we could get a model with hyper parameter optimalization that can perform better.

In [13]:
def data():
    #read the csv that the preproccessing notebok created
    df = pd.read_csv('cleaned.csv')

    # 20% test and 20% validation data
    train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
    # Save test data to csv to test the best model that we get with hyperoptimalization
    test.to_csv('test.csv', index=False)

    # target column is popularity
    x_train = train.drop(columns=['popularity'])
    y_train = train['popularity'].values

    x_test = validate.drop(columns=['popularity'])
    y_test = validate['popularity'].values

    # Not all features are standardized so let's do it before we start he training
    scaler = StandardScaler().fit(x_train)

    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)
    # save the scaler so we can use it later to standardize the test input                             )
    joblib.dump(scaler, 'scaler.save')
    return x_train, y_train, x_test, y_test

In [44]:
def create_model(x_train, y_train, x_test, y_test): 
    # number of hidden layers is a hyper parameter
    n_hidden = {{choice([1, 2, 3])}}
    layers = []
    dropouts = []
    for i in range(n_hidden):
        layers.append({{choice([30, 40, 50])}})
        dropouts.append({{uniform(0, 0.5)}})
    act = {{choice(['relu', 'leakyrelu'])}}
    optim = {{choice(['rmsprop', 'adam'])}}
    n_batch = {{choice([64, 128, 256])}}
    
    if act == 'relu':
        activation = keras.layers.ReLU()
    elif act == 'leakyrelu':
        activation = keras.layers.LeakyReLU()
    
    model = Sequential()
    # we have 50 features and at least one hidden layer
    model.add(Dense(layers[0], input_dim=50))
    model.add(activation)
    model.add(Dropout(dropouts[0]))
    # add layers based on hyper parameter
    for i in range(n_hidden - 1):
        model.add(Dense(layers[i + 1]))
        model.add(activation)
        model.add(Dropout(dropouts[i + 1]))
    # the output neuron
    model.add(Dense(output_dim=1, activation='relu'))
    
    model.compile(loss='mse', optimizer=optim)
    
    callbacks = [EarlyStopping(patience=30, verbose=0)]
    
    result = model.fit(x_train, y_train,
              batch_size=n_batch,
              epochs=100,
              verbose=2,
              validation_data=(x_test, y_test),
              callbacks=callbacks)

    best_loss = np.amin(result.history['val_loss']) 
    print('best loss:', best_loss)
    
    with open('hyperopt-log.csv', 'a') as csv_file:
        csv_file.write(str(n_hidden) + ';')
        for i in range(n_hidden):
            csv_file.write(str(layers[i]) + ';')
            csv_file.write(str(dropouts[i]) + ';')
        # write empty string if there are less than 3 layers
        for i in range(3 - n_hidden):
            csv_file.write(';;')
        csv_file.write(str(act) + ';')
        csv_file.write(str(optim) + ';')
        csv_file.write(str(n_batch) + ';')
        csv_file.write(str(best_loss) + '\n')
        
    return {'loss': best_loss, 'status': STATUS_OK, 'model': model}

In [45]:
import hyperas
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

In [46]:
with open('hyperopt-log.csv', 'w') as csv_file:
    csv_file.write('n_layers' + ';')
    csv_file.write('n_layer1' + ';')
    csv_file.write('dropout_1' + ';')
    csv_file.write('n_layer2' + ';')
    csv_file.write('dropout_2' + ';')
    csv_file.write('n_layer3' + ';')
    csv_file.write('dropout_3' + ';')
    csv_file.write('act' + ';')
    csv_file.write('optim' + ';')
    csv_file.write('n_batch' + ';')
    csv_file.write('best_loss' + '\n')

In [151]:
best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=20,
                                          notebook_name='train_and_hyperopt',
                                          trials=Trials())

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.preprocessing import StandardScaler
except:
    pass

try:
    from sklearn.externals import joblib
except:
    pass

try:
    from matplotlib import pyplot as plt
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.callbacks import EarlyStopping
except:
    pass

try:
    from keras.callbacks import ModelCheckpoint
except:
    pass

try:
    from keras.layers import Dense, Dropout, Activation
except:
    pass

try:
    from keras.models import load_model
except:
    pass

try:
    from keras.optimizers import SGD
except:
    pass

try:
    from sklearn.metrics import mean_squared_error
except:
    pass

try:
    import hyperas
except:
    pass

try:
    from hyperopt import Tria

/home/daniel/Documents/egyetem/deeplearning/hf/HitPredict/temp_model.py:152: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=1)`



Train on 103894 samples, validate on 34632 samples  
Epoch 1/100                                         
 - 4s - loss: 232.4902 - val_loss: 96.8962          

Epoch 2/100                                         
 - 3s - loss: 130.8948 - val_loss: 93.0149          

Epoch 3/100                                         
 - 3s - loss: 121.9264 - val_loss: 92.3435          

Epoch 4/100                                         
 - 3s - loss: 117.5643 - val_loss: 93.4623          

Epoch 5/100                                         
 - 4s - loss: 116.5988 - val_loss: 92.4094          

Epoch 6/100                                         
 - 4s - loss: 114.8329 - val_loss: 91.7855          

Epoch 7/100                                         
 - 4s - loss: 113.2393 - val_loss: 92.0743          

Epoch 8/100                                         
 - 3s - loss: 112.8278 - val_loss: 93.4752          

Epoch 9/100                                         
 - 3s - loss: 111.8681 - val_loss: 92.

Epoch 77/100                                        
 - 3s - loss: 98.0101 - val_loss: 91.1098           

Epoch 78/100                                        
 - 2s - loss: 97.6520 - val_loss: 91.0103           

Epoch 79/100                                        
 - 4s - loss: 97.5551 - val_loss: 91.1835           

Epoch 80/100                                        
 - 4s - loss: 97.4684 - val_loss: 91.1437           

Epoch 81/100                                        
 - 3s - loss: 97.6650 - val_loss: 90.8181           

Epoch 82/100                                        
 - 3s - loss: 97.4856 - val_loss: 91.1375           

Epoch 83/100                                        
 - 2s - loss: 97.4100 - val_loss: 91.0175           

Epoch 84/100                                        
 - 2s - loss: 97.3696 - val_loss: 90.9442           

Epoch 85/100                                        
 - 4s - loss: 97.3429 - val_loss: 90.9160           

Epoch 86/100                         

 - 2s - loss: 103.5827 - val_loss: 94.6021                                     

Epoch 35/100                                                                   
 - 3s - loss: 103.4588 - val_loss: 94.7952                                     

Epoch 36/100                                                                   
 - 2s - loss: 103.0562 - val_loss: 92.8140                                     

Epoch 37/100                                                                   
 - 4s - loss: 103.3124 - val_loss: 96.2167                                     

Epoch 38/100                                                                   
 - 2s - loss: 103.5229 - val_loss: 90.4361                                     

Epoch 39/100                                                                   
 - 2s - loss: 102.5028 - val_loss: 93.3167                                     

Epoch 40/100                                                                   
 - 1s - loss: 102.5762 - val_loss:

...

 - 2s - loss: 98.6626 - val_loss: 89.5183                                    

Epoch 33/100                                                                 
 - 2s - loss: 98.4836 - val_loss: 89.5566                                    

Epoch 34/100                                                                 
 - 2s - loss: 98.5716 - val_loss: 89.8519                                    

Epoch 35/100                                                                 
 - 2s - loss: 98.1773 - val_loss: 89.4622                                    

Epoch 36/100                                                                 
 - 3s - loss: 97.9097 - val_loss: 89.2732                                    

Epoch 37/100                                                                 
 - 2s - loss: 97.6312 - val_loss: 89.4224                                    

Epoch 38/100                                                                 
 - 3s - loss: 98.1278 - val_loss: 89.2979                 

Epoch 14/100                                                                 
 - 1s - loss: 247.7756 - val_loss: 244.8189                                  

Epoch 15/100                                                                 
 - 1s - loss: 247.2888 - val_loss: 244.8993                                  

Epoch 16/100                                                                 
 - 1s - loss: 247.1245 - val_loss: 244.8168                                  

Epoch 17/100                                                                 
 - 1s - loss: 247.6592 - val_loss: 244.8805                                  

Epoch 18/100                                                                 
 - 1s - loss: 247.4542 - val_loss: 245.0097                                  

Epoch 19/100                                                                 
 - 1s - loss: 247.0862 - val_loss: 244.7985                                  

Epoch 20/100                                              

Epoch 66/100                                                                 
 - 1s - loss: 245.3162 - val_loss: 244.7340                                  

Epoch 67/100                                                                 
 - 1s - loss: 245.1182 - val_loss: 244.5683                                  

Epoch 68/100                                                                 
 - 1s - loss: 245.1383 - val_loss: 244.5800                                  

Epoch 69/100                                                                 
 - 1s - loss: 245.2742 - val_loss: 244.7812                                  

Epoch 70/100                                                                 
 - 1s - loss: 245.2452 - val_loss: 244.6879                                  

Epoch 71/100                                                                 
 - 1s - loss: 245.1980 - val_loss: 244.6899                                  

Epoch 72/100                                              

 - 3s - loss: 88.7511 - val_loss: 89.5942                                    

Epoch 17/100                                                                 
 - 3s - loss: 88.7999 - val_loss: 89.4908                                    

Epoch 18/100                                                                 
 - 3s - loss: 88.6320 - val_loss: 90.5321                                    

Epoch 19/100                                                                 
 - 3s - loss: 88.6387 - val_loss: 89.8371                                    

Epoch 20/100                                                                 
 - 3s - loss: 88.4939 - val_loss: 90.8445                                    

Epoch 21/100                                                                 
 - 3s - loss: 88.4527 - val_loss: 91.7944                                    

Epoch 22/100                                                                 
 - 3s - loss: 88.5132 - val_loss: 89.2417                 

 - 3s - loss: 86.3367 - val_loss: 88.6630                                    

Epoch 69/100                                                                 
 - 3s - loss: 86.2608 - val_loss: 88.2110                                    

Epoch 70/100                                                                 
 - 4s - loss: 86.3195 - val_loss: 88.6185                                    

Epoch 71/100                                                                 
 - 3s - loss: 86.1454 - val_loss: 89.0332                                    

Epoch 72/100                                                                 
 - 4s - loss: 86.1827 - val_loss: 88.3846                                    

Epoch 73/100                                                                 
 - 4s - loss: 86.1640 - val_loss: 88.5298                                    

Epoch 74/100                                                                 
 - 4s - loss: 86.1213 - val_loss: 88.4627                 

In [152]:
test = pd.read_csv('test.csv')
x_test = test.drop(columns=['popularity'])
y_test = test['popularity'].values
scaler = joblib.load('scaler.save') 
x_test = scaler.transform(x_test)

In [153]:
best_model.evaluate(x_test, y_test)

34632/34632 [==============================] - 1s 19us/step


90.20560365792757